# Agency Prefix Code Profiles
Which agencies apply the most or have the most prefix codes? Which ones have the least? 

Data: 
* [Oblgiated list](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

* running notebook with updated locode data 'obligated_list_1202.csv'

In [34]:
import pandas as pd
from siuba import *

from IPython.display import Markdown
from IPython.core.display import display

from dla_utils import _dla_utils

/tmp/ipykernel_458/3076350911.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/dla_df.parquet")

In [3]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0,...,0.00,Large,0,0,1,0,0,0,0,1
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508,...,0.00,Large,0,0,0,0,0,1,0,1
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499,...,0.00,Large,0,0,0,0,0,1,0,1
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002,...,0.00,Large,1,0,0,0,0,1,0,2
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0,...,"6,130,295.48",Large,0,0,0,1,0,0,0,1


In [4]:
#making sure the locode and agency updated in the data.
## if updated, there will only be one agency listed.
## if multiple agencies listed, go back to add_locode.ipynb and run that notebook

df>>filter(_.locode=='6000')>>count(_.primary_agency_name)

,primary_agency_name,n
0,San Francisco Bay Area Rapid Transit District,43


## How many unique prefix codes per agency

### Top Agencies
finding the top agencies with the most unique prefix codes

In [5]:
top = _dla_utils.get_nunique(df, 'prefix', 'primary_agency_name')

In [6]:
top

,primary_agency_name,n
276,Los Angeles,38
277,Los Angeles County,32
434,San Bernardino Associated Governments,28
71,Caltrans,25
534,Stockton,25
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [7]:
_dla_utils.interactive_widget_counts(df,  'primary_agency_name', 'prefix')

Dropdown(description='Primary_Agency_Name', options=('Access Services', 'Agoura Hills', 'Alameda', 'Alameda - …

Output()

#### Printing top 5 programs for the top agencies


In [8]:
top

,primary_agency_name,n
276,Los Angeles,38
277,Los Angeles County,32
434,San Bernardino Associated Governments,28
71,Caltrans,25
534,Stockton,25
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [9]:
def find_agencies_top(agency=top.primary_agency_name.unique().tolist()):
    
    for agency in top.primary_agency_name.unique().tolist(): 
            count_top = ((df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(-_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_top)
            
    return

In [10]:
agency=top.primary_agency_name.unique().tolist()
find_agencies_top(agency)

**Top Prefix Codes Los Angeles uses:**

,prefix
30,STPL
23,HSIPL
11,CML
3,ATPL
6,BHLS


**Top Prefix Codes Los Angeles County uses:**

,prefix
12,ER
18,HSIPL
1,ACSTP
4,BPMPL
23,STPL


**Top Prefix Codes San Bernardino Associated Governments uses:**

...

,prefix
0,BRLS


**Top Prefix Codes Willows uses:**

,prefix
0,RPSTPL


**Top Prefix Codes Yountville uses:**

,prefix
0,RPSTPL


Popular codes in top agencies:
   * STP/STLN: Surface Transportation Program/Surface Transportation Program Local National Highway System
   * HSIP: Highway Safety Improvement Program 
   * CML: Congestion Mitigation & Air Quality Improvement Program 
   * BHLS: ?
   * BRLO: Highway Bridge Program (HBP) - Off Federal-aid System 

### Bottom Agencies

In [11]:
(df 
     >> group_by(_.primary_agency_name) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=1)
 
)

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
13,Amador City,1
16,Amador Rapid Transit System,1
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [12]:
df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
13,Amador City,1
16,Amador Rapid Transit System,1
...,...,...
587,Walnut Creek,3
592,West Covina,3
597,Westminster,3
605,Winters,3


In [13]:
bottom = df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

In [14]:
bottom.sample(9)

,primary_agency_name,n
217,Indian Wells,2
526,South Pasadena,2
554,Transit Joint Powers Authority For Merced County,3
446,San Dimas,1
235,Kings County Area Public Transit Agency,1
568,"U.S. Forest Service, Pacific Southwest Region",3
429,Sacramento Regional Transit District,3
212,Huntington Park,1
4,Alameda Corridor Transportation Authority,1


In [15]:
agency=bottom.primary_agency_name.unique().tolist()


def find_agencies_bottom(agency=bottom.primary_agency_name.unique().tolist()):
    
    for agency in bottom.primary_agency_name.unique().tolist(): 
            count_bottom = ((df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_bottom)
            
    return

In [16]:
print(len(agency))

283


In [17]:
find_agencies_bottom(agency)

**Top Prefix Codes Alameda Corridor Transportation Authority uses:**

,prefix
0,HPLUL


**Top Prefix Codes Alameda County Transportation Improvement Authority uses:**

,prefix
0,STPL


**Top Prefix Codes Alameda County Waste Management Authority uses:**

...

,prefix
2,STPL
0,ATPL
1,HSIPL


**Top Prefix Codes Winters uses:**

,prefix
1,HPLUL
0,BRLS
2,STPL


**Top Prefix Codes Yorba Linda uses:**

,prefix
0,HPLUL
2,STPL
1,HSIPL


Popular codes in bottom agencies: 
   * FTACML
   * STP/RPSTP
   * HSIP
   * CML 
   * BRLS
   * BRLO
   * CMLNI
    

* 140 agencies with 1 unique prefix code.
* 339 agencies with 3 or less prefix codes. 

## In which Caltrans Districts do these agencies have the most obligations?

### For the agencies in districts that applied the least

* joining the bottom agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [18]:
bottom = (df 
     >> group_by(_.primary_agency_name) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=3)
)

In [19]:
bottom_join = inner_join(df, bottom, on = "primary_agency_name")

In [20]:
bottom_dist = bottom_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [21]:
bottom_dist.rename(columns={'n': 'bottom_n'}, inplace=True)

In [22]:
bottom_dist

,dist,bottom_n
4,4,448
7,7,425
10,10,423
3,3,269
8,8,267
6,6,225
11,11,128
12,12,90
5,5,89
1,1,70


### For the agencies in districts that applied the most

* joining the top agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [23]:
top_join = inner_join(df, top, on = "primary_agency_name")

In [24]:
top_dist = top_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [25]:
top_dist.rename(columns={'n': 'top_n'}, inplace=True)

In [26]:
top_dist

,dist,top_n
4,4,3947
6,6,3676
7,7,3366
3,3,3021
10,10,2703
5,5,1786
8,8,1692
1,1,1557
11,11,1144
12,12,803


### Number of Obligations for Top and Bottom on District
 
* joining the dataframes we just made to compare the top and bottom easier
* finding which districts have the most obligations 

In [27]:
dist = full_join(bottom_dist, top_dist, on = "dist")

In [28]:
dist>>arrange(_.dist)

,dist,bottom_n,top_n
12,0,8,58
9,1,70,1557
10,2,53,782
3,3,269,3021
0,4,448,3947
8,5,89,1786
5,6,225,3676
1,7,425,3366
4,8,267,1692
11,9,13,313


In [29]:
dist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   dist      13 non-null     int64
 1   bottom_n  13 non-null     int64
 2   top_n     13 non-null     int64
dtypes: int64(3)
memory usage: 416.0 bytes


In [30]:
dist['bottom_n'] = dist['bottom_n'].fillna(0)

In [31]:
dist['bottom_n'] = dist['bottom_n'].astype(int)

In [32]:
dist.rename(columns={'dist': 'district'}, inplace=True)

In [33]:
dist>>arrange(_.district)

,district,bottom_n,top_n
12,0,8,58
9,1,70,1557
10,2,53,782
3,3,269,3021
0,4,448,3947
8,5,89,1786
5,6,225,3676
1,7,425,3366
4,8,267,1692
11,9,13,313


## Findings:



* **339 agencies** have **3 or less** unique prefixes. 
* **80 agencies** have **10 or more** unique prefixes, with the following **9 agenceies** listed with **20 or more**. 
* **District 7 and District 2** have the most obligations out of the top and bottom agencies 



| Top Agencies  | Bottom Agencies (sample of 9)|
| ------------- | ------------- | 
| Los Angeles | Sonoma County Transportation Authority | 
| Caltrans  |Port Of Long Beach |   
| Los Angeles County |Newport Beach |    
| San Bernardino Associated Governments | Portola | 
| San Diego | Calaveras Council Of Governments | 
| Riverside County | Hawthorne | 
| San Luis Obispo County | Corcoran |  
| Stockton | Claremont | 
| El Dorado County | El Cajon |  


